In [31]:
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [32]:
def show(data, width, height):
    image = numpy.array(data).reshape(width, height)
    plt.imshow(image, cmap=plt.cm.gray, interpolation="nearest")
    input('Press any key to continue!')

def save(fname, data, width, height):
    image = numpy.array(data).reshape(width, height)
    plt.imsave(fname, image, cmap=plt.cm.gray)

def view_weights(weights, width, height):
    cdict = {
        'red':
        [
            (0.0,  1.0, 1.0),
            (0.25,  1.0, 1.0),
            (0.5,  0.0, 0.0),
            (1.0,  0.0, 0.0)
        ],
        'green':
        [
            (0.0,  0.0, 0.0),
            (0.5,  0.0, 0.0),
            (0.75, 1.0, 1.0),
            (1.0,  1.0, 1.0)
        ],
        'blue':
        [
            (0.0,  0.0, 0.0),
            (1.0,  0.0, 0.0)
        ]
    }

    red_green = matplotlib.colors.LinearSegmentedColormap('green_red', cdict, 256)

    for i in range(0, 10):
        img = weights.flatten()[i::10].reshape((width, height))
        plt.imshow(img, cmap = red_green, clim=(-1, 1))
        if i == 0:
            pyplot.colorbar()
        input('Class ' + str(i))

In [33]:
from tensorflow.python.framework import dtypes, random_seed

#DataSet constructor, used to store an monochromatic image based dataset
#dtype can be either uint8 to leave the input as [0, 255], or float32 to rescale into [0, 1].
#Seed arg provides for convenient deterministic testing.
class DataSet(object):
    def __init__(self, images, labels, dtype = dtypes.float32, reshape = True, seed = None):
        seed1, seed2 = random_seed.get_seed(seed)

        #If op level seed is not set, use whatever graph level seed is returned
        numpy.random.seed(seed1 if seed is None else seed2)
        dtype = dtypes.as_dtype(dtype).base_dtype

        if dtype not in (dtypes.uint8, dtypes.float32):
            raise TypeError('Invalid image dtype %r, expected uint8 or float32' % dtype)

        assert images.shape[0] == labels.shape[0], ('images.shape: %s labels.shape: %s' % (images.shape, labels.shape))

        self.num_examples = images.shape[0]

        #Convert shape from [num examples, rows, columns, depth] to [num examples, rows*columns](assuming depth == 1)
        if reshape:
            assert images.shape[3] == 1
            images = images.reshape(images.shape[0], images.shape[1] * images.shape[2])

        #Convert from [0, 255] -> [0.0, 1.0].
        if dtype == dtypes.float32:
            images = images.astype(numpy.float32)
            images = numpy.multiply(images, 1.0 / 255.0)

        self.images = images
        self.labels = labels
        self.epochs_completed = 0
        self.index_in_epoch = 0

    #Return the next batch_size examples from this data set
    def next_batch(self, batch_size, shuffle=True):
        start = self.index_in_epoch

        #Shuffle for the first epoch
        if self.epochs_completed == 0 and start == 0 and shuffle:
            perm0 = numpy.arange(self.num_examples)
            numpy.random.shuffle(perm0)
            self.images = self.images[perm0]
            self.labels = self.labels[perm0]

        #Get Next epoch
        if start + batch_size > self.num_examples:
            #Finished epoch
            self.epochs_completed += 1

            #Get the rest examples in this epoch
            rest_num_examples = self.num_examples - start
            images_rest_part = self.images[start:self.num_examples]
            labels_rest_part = self.labels[start:self.num_examples]

            #Shuffle the data
            if shuffle:
                perm = numpy.arange(self.num_examples)
                numpy.random.shuffle(perm)
                self.images = self.images[perm]
                self.labels = self.labels[perm]

            #Start next epoch
            start = 0
            self.index_in_epoch = batch_size - rest_num_examples
            end = self.index_in_epoch
            images_new_part = self.images[start:end]
            labels_new_part = self.labels[start:end]
            return numpy.concatenate((images_rest_part, images_new_part), axis=0) , numpy.concatenate((labels_rest_part, labels_new_part), axis=0)
        else:
            self.index_in_epoch += batch_size
            end = self.index_in_epoch
            return self.images[start:end], self.labels[start:end]

In [34]:
import collections
import numpy
from random import shuffle
from tensorflow.python.framework import dtypes

Datasets = collections.namedtuple('Datasets', ['train', 'validation', 'test'])

def read_data_semeion(fname = 'data/semeion.data'):
    file = open(fname, 'r')
    lines = file.readlines()

    width = 16
    height = 16
    size = width * height
    classes = 10

    images = [];
    labels = [];
    fnumber = 0;

    for line in lines:
        data = line.split(' ')
        image = [];
        label = [];

        for i in range(0, size):
            image.append(int(float(data[i])))
        images.append(image)

        for i in range(size, size + classes):
            label.append(int(float(data[i]))) 
        labels.append(label)

        fnumber += 1
        #if fnumber < 10:
            #image_utils.show(image, width, height)
        #image_utils.save('./dataset/semeion/images/' + str(fnumber) + '.png', array, width, height)

    #Shuffle data
    images_shuffle = []
    labels_shuffle = []
    indexes = list(range(len(images)))
    shuffle(indexes)
    for i in indexes:
        images_shuffle.append(images[i])
        labels_shuffle.append(labels[i])

    images = images_shuffle
    labels = labels_shuffle

    samples = len(lines)
    train_samples = 1300
    test_samples = 1100

    #Train set
    image_train = numpy.array(images[:train_samples], dtype=numpy.uint8)
    image_train = image_train.reshape(train_samples, width, height, 1)

    label_train = numpy.array(labels[:train_samples], dtype=numpy.uint8)

    train = DataSet(image_train, label_train, reshape=True)

    #test set
    image_test = numpy.array(images[test_samples:], dtype=numpy.uint8)
    image_test = image_test.reshape(samples - test_samples, width, height, 1)

    label_test = numpy.array(labels[test_samples:], dtype=numpy.uint8)

    test = DataSet(image_test, label_test, reshape=True)

    return Datasets(train=train, validation=data, test=test)

In [37]:
#Dataset
width = 16
height = 16
dataset = read_data_semeion()

#Parameters
learning_rate = 0.001
iterations = 500
batch_size = 100

#Layers number of features
n_hidden_1 = 128
n_hidden_2 = 128

#Number of input and outputs
n_input = width * height
classes = 10

#Data placeholders
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, classes])

#Layers weights
w1 = tf.Variable(tf.random_normal([n_input, n_hidden_1]))
w2 = tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]))
w_out = tf.Variable(tf.random_normal([n_hidden_2, classes]))

#Layers bias
b1 = tf.Variable(tf.random_normal([n_hidden_1]))
b2 = tf.Variable(tf.random_normal([n_hidden_2]))
b_out = tf.Variable(tf.random_normal([classes]))

#Hidden layer 1
z_1 = tf.add(tf.matmul(x, w1), b1)
a_1 = tf.nn.sigmoid(z_1)

#Hidden layer 2
z_2 = tf.add(tf.matmul(a_1, w2), b2)
a_2 = tf.nn.sigmoid(z_2)

#Output layer with linear activation
pred = tf.matmul(a_2, w_out) + b_out


diff = tf.subtract(a_2, y)

def sigmaprime(x):
    return tf.multiply(tf.nn.sigmoid(x), tf.subtract(tf.constant(1.0), tf.nn.sigmoid(x)))


d_z_2 = tf.multiply(diff, sigmaprime(a_2))
d_b_2 = d_z_2
d_w_2 = tf.matmul(tf.transpose(a_1), d_z_2)

d_a_1 = tf.matmul(d_z_2, tf.transpose(w2))
d_z_1 = tf.multiply(d_a_1, sigmaprime(z_1))
d_b_1 = d_z_1
d_w_1 = tf.matmul(tf.transpose(x), d_z_1)

acct_mat = tf.equal(tf.argmax(a_2, 1), tf.argmax(y, 1))
acct_res = tf.reduce_sum(tf.cast(acct_mat, tf.float32))

#Cost and optimizer
#cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
#optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

#Start time
start = time.time()

#Session
init = tf.global_variables_initializer()
session = tf.Session()
session.run(init)

#Training cycle
for i in range(iterations):
    total_batch = int(dataset.train.num_examples / batch_size)

    #Loop over all batches
    for i in range(total_batch):
        batch_x, batch_y = dataset.train.next_batch(batch_size)

        session.run(acct_res, feed_dict={x: batch_x, y: batch_y})

    #Test
    if i % 5 == 0:
        correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        print("Iteration: " + str(i) + " Accuracy: ", accuracy.eval(session=session, feed_dict={x: dataset.test.images, y: dataset.test.labels}))

#End time
end = time.time()

#Test
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
print("Accuracy:", accuracy.eval(session=session, feed_dict={x: dataset.test.images, y: dataset.test.labels}))

#Time
print('Time: ' + str(end - start));

ValueError: Dimensions must be equal, but are 128 and 10 for 'Sub_6' (op: 'Sub') with input shapes: [?,128], [?,10].